### Relevante pakker

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm

ModuleNotFoundError: No module named 'statsmodels'

Merk: ChatGPT har blitt brukt for å få hjelp til å skrive deler av denne koden.

### Rense dataene

In [ ]:
df = pd.read_csv("Data/lego.population.csv", sep = ",", encoding = "latin1")
df

In [ ]:
# fjerner forklaringsvariabler vi ikke trenger
df2 = df[['Set_Name', 'Theme', 'Pieces', 'Price', 'Pages', 'Minifigures', 'Unique_Pieces', 'Ages']]

# fjerner observasjoner med manglende datapunkter
df2 = df2.dropna()

# gjør themes om til string og fjern alle tegn vi ikke vil ha med
df2['Theme'] = df2['Theme'].astype(str)
df2['Theme'] = df2['Theme'].str.replace(r'[^a-zA-Z0-9\s-]', '', regex = True)

# fjerner dollartegn og trademark-tegn fra datasettet
df2['Price'] = df2['Price'].str.replace('\$', '', regex = True)

# og gjør så prisen om til float
df2['Price'] = df2['Price'].astype(float)

df2

In [ ]:
def categorize_age_from_theme(theme):
    if theme in ['DUPLO', 'Classic', 'Juniors']:
        return 'Småbarn'
    elif theme in ['Friends', 'City', 'Unikitty', 'NINJAGO', 'Minecraft', 'Batman', 'THE LEGO MOVIE 2', 'Speed Champions',
                   'Harry Potter', 'Hidden Side', 'LEGO Frozen 2', 'Trolls World Tour', 'Minions', 'Powerpuff Girls',
                   'Jurassic World', 'Overwatch', 'Spider-Man', 'Creator 3-in-1', 'Monkie Kid', 'Powered UP']:
        return 'Skolebarn'
    elif theme in ['Star Wars', 'Marvel', 'DC', 'Ideas', 'Creator Expert', 'Stranger Things']:
        return 'Eldre'
    elif theme in ['Disney', 'Minifigures']:
        return 'Uvisst'
    else:
        print("Unknown age group: " + theme)
        return 'Unknown'  # For aldersgrupper som ikke passer inn i de definerte kategoriene
    
def categorize_age_group(age_group):
    if age_group in ['Ages_1½+', 'Ages_2+', 'Ages_2-5']:
        return 'Småbarn'
    elif age_group in ['Ages_4+', 'Ages_5+', 'Ages_6+', 'Ages_4-7', 'Ages_5-10', 'Ages_6-10', 'Ages_5-99', 'Ages_7+', 'Ages_8+', 'Ages_7-12', 'Ages_8-12', 'Ages_7-14', 'Ages_8-14', 'Ages_5-12', 'Ages_6-12', 'Ages_6-14']:
        return 'Skolebarn'
    elif age_group in ['Ages_9+', 'Ages_10+', 'Ages_12+', 'Ages_9-14']:
        return 'Eldre_skolebarn'
    elif age_group in ['Ages_14+', 'Ages_16+', 'Ages_18+']:
        return 'Eldre'
    else:
        print("Unknown age group: " + age_group)
        return 'Unknown'  # For aldersgrupper som ikke passer inn i de definerte kategoriene

df2['Age_Category_Ages'] = df2['Ages'].apply(categorize_age_group)
df2['Age_Category'] = df2['Theme'].apply(categorize_age_from_theme)

### Mer eller mindre relevante kodesnutter

In [ ]:
print(df2.mean(numeric_only = True))

In [ ]:
print(df2['Theme'].value_counts())
# print(df2['Age_Category'].value_counts())

In [ ]:
print(df2['Theme'].unique())

In [ ]:
plt.hist(df2['Price'], bins=20, color='skyblue', edgecolor='black')
plt.xlabel('Pris i dollar [$]')
plt.ylabel('')
plt.gca().set_aspect(1)
plt.show()

In [ ]:
Theme_order = df2['Theme'].value_counts().index

# Set a larger figure size
plt.figure(figsize=(15, 7))

# Assuming df is your DataFrame with the 'Ages' column
sns.countplot(data=df2, x='Theme', order=Theme_order)

# Rotate x-axis labels for better readability
plt.xticks(rotation=70)  # You can adjust the angle as needed

# Make x-axis label box taller to fit the rotated labels
# plt.tight_layout()

# Adjust plot aesthetics if needed
plt.xlabel('Theme')
plt.ylabel('Count')
plt.title('Count of Lego sets per Theme')

# Show the plot
plt.show()

In [ ]:
Age_Category_order = df2['Age_Category'].value_counts().index

# Set a larger figure size
plt.figure(figsize=(12, 6))

# Assuming df is your DataFrame with the 'Ages' column
sns.countplot(data=df2, x='Age_Category', order=Age_Category_order)

# Rotate x-axis labels for better readability
# plt.xticks(rotation=45)  # You can adjust the angle as needed

# Adjust plot aesthetics if needed
plt.xlabel('Age Categories')
plt.ylabel('Count')
plt.title('Count of Lego sets per Age Category')

# Show the plot
plt.show()

In [ ]:
# Print all the Themes for each Age Category
for age_category in df2['Age_Category'].unique():
    print(age_category + ': ' + str(df2[df2['Age_Category'] == age_category]['Theme'].unique()))

In [ ]:
plt.scatter(df2['Pieces'], df2['Price'])
plt.xlabel('Antall brikker')
plt.ylabel('Pris i dollar [$]')
plt.gca().set_aspect(5)
plt.show()

In [ ]:
# hva er det dyreste settet i datasettet mon tro?
print(df2.loc[df2['Price'].idxmax()])

In [ ]:
# og hvilket har flest brikker?
print(df2.loc[df2['Pieces'].idxmax()])

In [ ]:
# hvilke tema har de billigste settene?
df2.groupby('Theme')['Price'].mean().sort_values(ascending=False)#[:3]

In [ ]:
result = df2.groupby('Age_Category')['Ages'].value_counts().rename('count').reset_index()

filtered_result = result[result['count'] > 1].sort_values(by=['Age_Category', 'count'], ascending=[True, False])

print(filtered_result)

In [ ]:
# hvilke tema har flest brikker?
Age_Pieces = df2.groupby('Age_Category')['Pieces'].mean().sort_values(ascending=False)#[:5]

print('Average number of pieces per Age Category')
print()
print(Age_Pieces.reset_index().to_string(index=False, col_space=15))

In [ ]:
# Print the prices and Age_Category in a list with headers
Age_Price = df2.groupby('Age_Category')['Price'].mean().sort_values(ascending=False)

print('Average price per Age_Category:')
print()
print(Age_Price.reset_index().to_string(index=False, col_space=15))

In [ ]:
Age_UniquePieces = df2.groupby('Age_Category')['Unique_Pieces'].mean().sort_values(ascending=False)

print('Average number of unique pieces per Age_Category:')
print()
print(Age_UniquePieces.reset_index().to_string(index=False, col_space=15))

In [ ]:
Age_Minifigures = df2.groupby('Age_Category')['Minifigures'].mean().sort_values(ascending=False)

print('Average number of minifigures per Age_Category:')
print()
print(Age_Minifigures.reset_index().to_string(index=False, col_space=15))

In [ ]:
Age_Pages = df2.groupby('Age_Category')['Pages'].mean().sort_values(ascending=False)

print('Average number of pages per Age_Category:')
print()
print(Age_Pages.reset_index().to_string(index=False, col_space=15))

In [ ]:
# Selecting columns for correlation
columns_of_interest = ['Price', 'Pieces', 'Unique_Pieces', 'Minifigures', 'Pages']

# Creating a subset of the DataFrame with selected columns
subset_df = df2[columns_of_interest]

# Calculating the correlation matrix
correlation_matrix = subset_df.corr()

# Displaying the correlation matrix
print(correlation_matrix)

# Visualizing the correlation matrix using a heatmap
display(correlation_matrix.style.background_gradient(cmap = 'coolwarm', axis = None, vmin = -1, vmax = 1))
plt.show()

In [ ]:
# Selecting columns for correlation
columns_of_interest = ['Price', 'Pieces', 'Unique_Pieces', 'Pages', 'Minifigures', 'Age_Category']

# Creating a subset of the DataFrame with selected columns
subset_df = df2[columns_of_interest]

# Creating dummy variables for 'Age_Category'
dummy_age_category = pd.get_dummies(subset_df['Age_Category'], prefix='Age_Category')

# Concatenating dummy variables with the original DataFrame
subset_df = pd.concat([subset_df, dummy_age_category], axis=1)

# Dropping the original 'Age_Category' column as it's no longer needed
subset_df = subset_df.drop('Age_Category', axis=1)

# Calculating the correlation matrix
correlation_matrix = subset_df.corr()

# Displaying the correlation matrix with a background gradient
display(correlation_matrix.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1))
plt.show()

In [ ]:
# Selecting columns for correlation
columns_of_interest = ['Price', 'Pieces', 'Unique_Pieces', 'Pages', 'Minifigures', 'Age_Category_Ages']

# Creating a subset of the DataFrame with selected columns
subset_df = df2[columns_of_interest]

# Creating dummy variables for 'Age_Category'
dummy_age_category = pd.get_dummies(subset_df['Age_Category_Ages'], prefix='Age_Category')

# Concatenating dummy variables with the original DataFrame
subset_df = pd.concat([subset_df, dummy_age_category], axis=1)

# Dropping the original 'Age_Category' column as it's no longer needed
subset_df = subset_df.drop('Age_Category_Ages', axis=1)

# Calculating the correlation matrix
correlation_matrix = subset_df.corr()

# Displaying the correlation matrix with a background gradient
display(correlation_matrix.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1))
plt.show()

In [ ]:
sns.pairplot(df2, vars = ['Price', 'Pieces', 'Unique_Pieces', 'Pages', 'Minifigures'],
             hue = 'Age_Category', 
             diag_kind = 'kde',
             plot_kws = dict(alpha = 0.4))
plt.show()

In [ ]:
df4 = df2[['Price', 'Pieces', 'Unique_Pieces', 'Pages', 'Minifigures', 'Age_Category']]

# Remove all rows with Age_Category = 'Uvisst' or 'Unknown'
df4 = df4[df4['Age_Category'] != 'Uvisst']
df4 = df4[df4['Age_Category'] != 'Unknown']
df4 = pd.get_dummies(df4, prefix = 'Age_Category', columns = ['Age_Category'])

# enkel lineær regresjon
formel = 'Price ~ Pieces + Unique_Pieces + Pages + Minifigures + Age_Category_Skolebarn + Age_Category_Småbarn + Age_Category_Eldre'

modell = smf.ols(formel, data = df4)
resultat = modell.fit()

resultat.summary()

In [ ]:
df4 = df2[['Price', 'Pieces', 'Unique_Pieces', 'Pages', 'Minifigures', 'Age_Category']]

# Remove all rows with Age_Category = 'Uvisst' or 'Unknown'
df4 = df4[df4['Age_Category'] != 'Uvisst']
df4 = df4[df4['Age_Category'] != 'Unknown']

# Convert Age_Category to numerical values for regression analysis
df4['Age_Category_Num'] = df4['Age_Category'].map({'Småbarn': 1, 'Skolebarn': 2, 'Eldre': 3})

# Fit the regression model using OLS
X = sm.add_constant(df4[['Age_Category_Num']])
y = df4['Price']
model = sm.OLS(y, X).fit()

# Print the regression summary
print(model.summary())

In [ ]:
df4 = df2[['Price', 'Pieces', 'Unique_Pieces', 'Pages', 'Minifigures', 'Age_Category_Ages']]

# Remove all rows with Age_Category = 'Uvisst' or 'Unknown'
df4 = df4[df4['Age_Category_Ages'] != 'Uvisst']
df4 = df4[df4['Age_Category_Ages'] != 'Unknown']

df4 = pd.get_dummies(df4, prefix = 'Age_Category', columns = ['Age_Category_Ages'])

# enkel lineær regresjon
formel = 'Price ~ Pieces + Unique_Pieces + Pages + Minifigures + Age_Category_Skolebarn + Age_Category_Småbarn + Age_Category_Eldre + Age_Category_Eldre_skolebarn'

modell = smf.ols(formel, data = df4)
resultat = modell.fit()

resultat.summary()

In [ ]:
slope = resultat.params['Pieces']
intercept = resultat.params['Intercept']

regression_x = np.array(df2['Pieces'])

regression_y = slope * regression_x + intercept

plt.scatter(df2['Pieces'], df2['Price'], label='Data Points')
plt.plot(regression_x, regression_y, color='red', label='Regression Line')
plt.xlabel('Antall brikker')
plt.ylabel('Pris [$]')
plt.title('Kryssplott med regresjonslinje (enkel LR)')
plt.legend()
plt.grid()
plt.show()

In [ ]:
figure, axis = plt.subplots(1, 2, figsize = (15, 5))
sns.scatterplot(x = resultat.fittedvalues, y = resultat.resid, ax = axis[0])
axis[0].set_ylabel("Residual")
axis[0].set_xlabel("Predikert verdi")

sm.qqplot(resultat.resid, line = '45', fit = True, ax = axis[1])
axis[1].set_ylabel("Kvantiler i residualene")
axis[1].set_xlabel("Kvantiler i normalfordelingen")
plt.show()

In [ ]:
# mythemes = ['Star Wars', 'NINJAGO', 'Harry Potter']
# subset_df = df2[df2['Theme'].isin(mythemes)]
subset_df = df2
sns.pairplot(subset_df, vars = ['Price', 'Pieces', 'Pages', 'Minifigures', 'Unique_Pieces'],
             hue = 'Age_Category', 
             diag_kind = 'kde',
             plot_kws = dict(alpha = 0.4))
plt.show()

In [ ]:
# enkel lineær regresjon, tar ikke hensyn til tema
res_sub = smf.ols('Price ~ Pieces' , data = subset_df).fit()

In [ ]:
# enkel LR for hvert tema hver for seg
resultater = []
ageCategories = subset_df['Age_Category'].unique()
for i, ageCategory in enumerate(ageCategories):
    modell3 = smf.ols('Price ~ Pieces' , data = subset_df[subset_df['Age_Category'].isin([ageCategory])])
    resultater.append(modell3.fit())

In [ ]:
# plott av dataene og regresjonslinjene
for i, ageCategory in enumerate(ageCategories):
    slope = resultater[i].params['Pieces']
    intercept = resultater[i].params['Intercept']

    regression_x = np.array(subset_df[subset_df['Age_Category'].isin([ageCategory])]['Pieces'])
    regression_y = slope * regression_x + intercept

    # Plot scatter plot and regression line
    plt.scatter(subset_df[subset_df['Age_Category'].isin([ageCategory])]['Pieces'], subset_df[subset_df['Age_Category'].isin([ageCategory])]['Price'], color=plt.cm.tab10(i))
    plt.plot(regression_x, regression_y, color=plt.cm.tab10(i), label=ageCategory)
    
plt.xlabel('Antall brikker')
plt.ylabel('Pris')
plt.title('Kryssplott med regresjonslinjer')
plt.legend()
plt.grid()
plt.show()

In [ ]:
print(df2['Age_Category'].unique())

In [ ]:
# Konverter Age_Category til dummy-variabler
df3 = df2.copy()
df3['Age_Category'].fillna('missing', inplace=True)
df3 = pd.get_dummies(df3, columns=['Age_Category'], drop_first=True)

# Velg forklaringsvariabler og responsvariabel
X = df3.drop(['Price', 'Set_Name', 'Theme', 'Unique_Pieces', 'Pages', 'Minifigures', 'Ages', 'Pieces'], axis=1)
y = df3['Price']

# Print data types of X for verification
print(X.dtypes)
print(y.dtypes)
print(X.value_counts())

columns_to_fill = ['Age_Category_Older_Children', 'Age_Category_Teenagers_and_Adults', 'Age_Category_Toddler', 'Age_Category_Young_Children']

# Fill missing values with 0 for the specified columns
df3[columns_to_fill] = df3[columns_to_fill].fillna(0)

df3[columns_to_fill] = df3[columns_to_fill].replace('', False)
df3[columns_to_fill] = df3[columns_to_fill].fillna(False)

for column in columns_to_fill:
    df[df[column] != True]
    if (df3[column].any()):
        print(column, df3[column].unique())

# Legg til en konstant til X (nødvendig for statsmodels)
X = sm.add_constant(X)

# Opprett og tilpasse modellen
model = sm.OLS(y, X).fit()

# Print ut en oppsummering av modellen
print(model.summary())


In [ ]:
# multippel lineær regresjon
modell3_mlr = smf.ols('Price ~ Pieces + Theme' , data = subset_df)
modell3_mlr.fit().summary()

In [ ]:
# multippel lineær regresjon med en annen referansekategori
modell3_mlr_alt = smf.ols('Price ~ Pieces + C(Theme, Treatment("Star Wars"))' , data = subset_df)
modell3_mlr_alt.fit().summary()

In [ ]:
# plott
intercept = [modell3_mlr.fit().params['Theme[T.Star Wars]'], modell3_mlr.fit().params['Theme[T.NINJAGO]'], 0] + modell3_mlr.fit().params['Intercept']
slope = modell3_mlr.fit().params['Pieces']

for i, theme in enumerate(mythemes):

    regression_x = np.array(subset_df[subset_df['Theme'].isin([theme])]['Pieces'])
    regression_y = slope * regression_x + intercept[i]

    # Plot scatter plot and regression line
    plt.scatter(subset_df[subset_df['Theme'].isin([theme])]['Pieces'], subset_df[subset_df['Theme'].isin([theme])]['Price'], color = plt.cm.tab10(i))
    plt.plot(regression_x, regression_y, color=plt.cm.tab10(i), label=theme)
    
# uten tema som forklaringsvariabel:
regression_x = np.array(subset_df['Pieces'])
regression_y = res_sub.params['Pieces'] * regression_x + res_sub.params['Intercept']
plt.plot(regression_x, regression_y, color='black', label = 'No theme')
    
plt.xlabel('Antall brikker')
plt.ylabel('Pris')
plt.title('Kryssplott med regresjonslinjer')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# med interaksjonsledd mellom antall brikker og tema
modell3_mlri = smf.ols('Price ~ Pieces*Theme' , data = subset_df)
modell3_mlri.fit().summary()

In [ ]:
# plott
intercept = [modell3_mlri.fit().params['Theme[T.Star Wars]'], modell3_mlri.fit().params['Theme[T.NINJAGO]'], 0] + modell3_mlri.fit().params['Intercept']
slope = [modell3_mlri.fit().params['Pieces:Theme[T.Star Wars]'], modell3_mlri.fit().params['Pieces:Theme[T.NINJAGO]'], 0] + modell3_mlri.fit().params['Pieces']

for i, theme in enumerate(mythemes):

    regression_x = np.array(subset_df[subset_df['Theme'].isin([theme])]['Pieces'])
    regression_y = slope[i] * regression_x + intercept[i]

    # Plot scatter plot and regression line
    plt.scatter(subset_df[subset_df['Theme'].isin([theme])]['Pieces'], subset_df[subset_df['Theme'].isin([theme])]['Price'], color=plt.cm.tab10(i))
    plt.plot(regression_x, regression_y, color=plt.cm.tab10(i), label=theme)
    
# uten tema som forklaringsvariabel:
regression_x = np.array(subset_df['Pieces'])
regression_y = res_sub.params['Pieces'] * regression_x + res_sub.params['Intercept']
plt.plot(regression_x, regression_y, color='black', label='Theme unaccounted for')
    
plt.xlabel('Antall brikker')
plt.ylabel('Pris [$]')
plt.title('Kryssplott med regresjonslinjer')
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Kode for å lagre plot som (.png)
## fjern 'plt.show()' og erstatt med:
# plt.savefig('my_plot.png')

In [ ]:
# Steg 5: Evaluere om modellen passer til dataene
# Plotte predikert verdi mot residual
figure, axis = plt.subplots(1, 2, figsize = (15, 5))
sns.scatterplot(x = modell3_mlri.fit().fittedvalues, y = modell3_mlri.fit().resid, ax = axis[0])
axis[0].set_ylabel("Residual")
axis[0].set_xlabel("Predikert verdi")

# Lage kvantil-kvantil-plott for residualene
sm.qqplot(modell3_mlri.fit().resid, line = '45', fit = True, ax = axis[1])
axis[1].set_ylabel("Kvantiler i residualene")
axis[1].set_xlabel("Kvantiler i normalfordelingen")
plt.show()

In [ ]:
# Gruppere temaer i nye grupper:
# (Harry Potter, NINJAGO og Star Wars havner i én gruppe, City og Friends i en annen, og alle andre i en tredje)
df2['cat'] = np.where(df2['Theme'].isin(['Harry Potter', 'NINJAGO', 'Star Wars']), 'Cat1', 
                      np.where(df2['Theme'].isin(['City', 'Friends']), 'Cat2', 'Cat3'))
df2.groupby(['cat']).size().reset_index(name = 'Count')

In [ ]:
df2.groupby(['cat', 'Theme']).size().reset_index(name = 'Count')


In [ ]:
# Importing necessary libraries
import pandas as pd
from sklearn.linear_model import LinearRegression

# Path to the dataset
file_population_path = '/Data/lego.population.csv'

# Reading the dataset with ISO-8859-1 encoding
lego_population = pd.read_csv(file_population_path, encoding='ISO-8859-1')

# Converting the Price column to numeric, removing any non-numeric characters
lego_population['Price'] = pd.to_numeric(lego_population['Price'].replace('[\$,]', '', regex=True), errors='coerce')

# Defining the age categories based on themes
age_categories = {
    'Skolebarn': ['Friends', 'City', 'Unikitty', 'NINJAGO', 'Minecraft', 'Batman',
                  'THE LEGO MOVIE 2', 'Speed Champions', 'Harry Potter', 'Hidden Side',
                  'LEGO Frozen 2', 'Trolls World Tour', 'Minions', 'Powerpuff Girls',
                  'Jurassic World', 'Overwatch', 'Spider-Man', 'Creator 3-in-1', 'Monkie Kid',
                  'Powered UP'],
    'Uvisst': ['Disney', 'Minifigures'],
    'Småbarn': ['DUPLO', 'Classic', 'Juniors'],
    'Eldre': ['Star Wars', 'Marvel', 'DC', 'Ideas', 'Creator Expert', 'Stranger Things']
}

# Assigning age categories to the LEGO sets in the dataset
lego_population['Age_Category'] = lego_population['Theme'].apply(lambda theme: next((category for category, themes in age_categories.items() if theme in themes), 'Unknown'))

# Preparing dataset for regression analysis, including age categories
regression_dataset_with_age = lego_population[['Price', 'Pages', 'Pieces', 'Unique_Pieces', 'Minifigures', 'Age_Category']].dropna()

# One-hot encoding for age categories
regression_dataset_with_age = pd.get_dummies(regression_dataset_with_age, columns=['Age_Category'], drop_first=True)

# Defining the dependent variable (y) and independent variables (X) including age categories
y_with_age = regression_dataset_with_age['Price']
X_with_age = regression_dataset_with_age.drop('Price', axis=1)

# Creating the Linear Regression model
model_with_age = LinearRegression()

# Fitting the model with the dataset
model_with_age.fit(X_with_age, y_with_age)

# Getting the coefficients and intercept of the model
coefficients_with_age = model_with_age.coef_
intercept_with_age = model_with_age.intercept_

# Creating a dictionary to map the coefficients to their corresponding variables
variable_names = X_with_age.columns.tolist()
coefficients_dict = dict(zip(variable_names, coefficients_with_age))

# Adding the intercept to the dictionary
coefficients_dict['Intercept'] = intercept_with_age

# Displaying the detailed results
coefficients_dict
